## 闭包

In [9]:
def callf(func):
    x=14
    return func()

In [2]:
def helloworld():
    return 'Hello world'

In [3]:
callf(helloworld)

'Hello world'

In [175]:
def bar():
    x = 13
    def HelloWorld():
        return 'Hello World. x is %d' % x
    return callf(HelloWorld)
bar()

'Hello World. x is 13'

In [28]:
import requests
def page(url):
    def get():
        r = requests.get(url)
        r.encoding='utf-8'
        return r.text
    return get

In [29]:
sina = page("http://www.sina.com.cn/")
sina()

'<!DOCTYPE html>\n<!-- [ published at 2018-07-21 16:10:00 ] -->\n<html>\n<head>\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <title>新浪首页</title>\n\t<meta name="keywords" content="新浪,新浪网,SINA,sina,sina.com.cn,新浪首页,门户,资讯" />\n\t<meta name="description" content="新浪网为全球用户24小时提供全面及时的中文资讯，内容覆盖国内外突发新闻事件、体坛赛事、娱乐时尚、产业资讯、实用信息等，设有新闻、体育、娱乐、财经、科技、房产、汽车等30多个内容频道，同时开设博客、视频、论坛等自由互动交流空间。" />\n\t<meta content="always" name="referrer">\n    <link rel="mask-icon" sizes="any" href="//www.sina.com.cn/favicon.svg" color="red">\n\t<meta name="stencil" content="PGLS000022" />\n\t<meta name="publishid" content="30,131,1" />\n\t<meta name="verify-v1" content="6HtwmypggdgP1NLw7NOuQBI2TW8+CfkYCoyeB8IDbn8=" />\n\t<meta name="360-site-verification" content="63349a2167ca11f4b9bd9a8d48354541" />\n\t<meta name="application-name" content="新浪首页"/>\n\t<meta name ="msapplication-TileImage" content="//i1.sinaimg.cn/dy/deco/2013/0

In [30]:
def countdown(n):
    def nextn():
        nonlocal n
        r = n
        n -= 1
        return r
    return nextn

In [33]:
nextn = countdown(10)
nextn()

10

## 装饰器

### 1.简单装饰器

In [43]:
def trace(func):
    enable_tracing = True
    if enable_tracing:
        debug_log = open("debug.log",'w')
        def callf(*args,**kwargs):
            debug_log.write("Calling %s : %s,%s\n"%
                            (func.__name__,args,kwargs))
            r = func(*args,**kwargs)
            debug_log.write("%s returned %s \n"%(func.__name__,r))
            return r
        return callf
    else:
        return func

In [53]:
@trace
def square(x):
    return x*x

In [54]:
square(2)

4

In [115]:
def foo(func):
    def wrapper(*args):
        print("foo:{}".format(args))
        return func(*args)
    return wrapper

In [116]:
def bar(func):
    def wrapper(*args):
        print("bar:{}".format(args))
        return func(*args)
    return wrapper

In [121]:
def spam(func):
    def wrapper(*args):
        print("spam:{}".format(args))
        return(func(*args))
    return wrapper

In [125]:
@foo
@bar
@spam
def grok(x):
    return "grok:%s"%x

In [123]:
grok('hello')

foo:('hello',)
bar:('hello',)
spam:('hello',)


'grok:hello'

In [136]:
def debug(func):
    def wrapper(*args,**kwargs):
        print("[DEBUG]:enter {}()".format(func.__name__))
        return func(*args,**kwargs)
    return wrapper

In [138]:
@debug
def say(something):
    print("hello {}!".format(something))

In [139]:
say('你好')

[DEBUG]:enter say()
hello 你好!


### 2.带参装饰器

In [210]:
even_handlers = {}
def eventhandler(event):
    def register_function(f):
        even_handlers[event] = f.__name__
        return f
    return register_function

In [211]:
@eventhandler('BUTTON')
def handle_button(msg):
    return "hello,%s"%msg

In [212]:
handle_button('world')

'hello,world'

In [213]:
even_handlers

{'BUTTON': 'handle_button'}

In [159]:
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args,**kwargs):
            print("[{level}]:enter function {func}()".format(
            level = level,func=func.__name__))
            return func(*args,**kwargs)
        return inner_wrapper
    return wrapper

In [160]:
@logging(level = 'INFO')
def say(something):
    print("say {}".format(something))

In [161]:
say('world')

[INFO]:enter function say()
say world


In [162]:
@logging(level = 'DEBUG')
def do(something):
    print("do {}...".format(something))
    

In [163]:
do("debug")

[DEBUG]:enter function do()
do debug...


### 3.类装饰器

In [164]:
class Test:
    def __call__(self):
        print('call me')

In [165]:
t= Test()
t()

call me


In [168]:
class Logging:
    def __init__(self,func):
        self.func = func
    def __call__(self,*args,**kwargs):
        print("[DEBUG]:enter function {func}()".format(
        func = self.func.__name__))
        return self.func(*args,**kwargs)

In [169]:
@Logging
def greeting(something):
    print("say {}!".format(something))

In [170]:
greeting('hi')

[DEBUG]:enter function greeting()
say hi!


In [171]:
class Logging1:
    def __init__(self,level='INFO'):
        self.level=level
    def __call__(self,func):#接受函数
        def wrapper(*args,**kwargs):
            print("[{level}]:enter function {func}()".format(
            level = self.level,func = func.__name__))
            func(*args,**kwargs)
        return wrapper

In [172]:
@Logging1(level='DEBUG')
def do(sth):
    print("do {}".format(sth))

In [173]:
do('debug')

[DEBUG]:enter function do()
do debug


In [183]:
class A:
    bar = 1
    def __init__(self,name,age):
        self.name = name
        self.age = age
        
    def foo(self):
        print("call function foo in class A")
    
    @staticmethod
    def static_foo():
        print('static foo')
        print(A.bar)
        
    @classmethod
    def class_foo(cls):
        print('class foo')
        print(cls.bar)
        cls().foo()
    
    @property
    def property_foo(self):
        print(self.name)
        return('yes')

In [184]:
A.static_foo()

static foo
1


In [186]:
num = A("liu","21")
num.property_foo

liu


'yes'

## 生成器

In [187]:
def countdown(n):
    print('Counting down from %d'%n)
    while n > 0:
        yield n
        n -= 1
    return

In [188]:
c = countdown(10)

In [190]:
c.__next__()

Counting down from 10


10

In [191]:
c.__next__()

9

In [192]:
def receiver():
    print("ready to receive")
    while True:
        n = (yield)
        print("Got %s"%n)

In [193]:
r = receiver()

In [194]:
r.__next__()

ready to receive


In [195]:
r.send(1)

Got 1


In [196]:
r.send(2)

Got 2


In [197]:
r.send("hello")

Got hello


In [201]:
def coroutine(func):
    def start(*args,**kwargs):
        g = func(*args,**kwargs)
        g.__next__()
        return g
    return start

In [202]:
@coroutine
def receiver():
    print("ready to receive")
    while True:
        n = (yield)
        print("Got %s"%n)

In [205]:
r = receiver()
r.send('hello')

ready to receive
Got hello


In [206]:
r.close()

In [207]:
r.send(4)

StopIteration: 

In [13]:
lines = open("portfolio.txt")

In [14]:
fields = (line.split() for line in lines)

In [15]:
sum(float(f[1])*float(f[2]) for f in fields)

44671.15

In [16]:
import math

In [17]:
a = eval('3*math.sin(3.5) + 7.2')

In [18]:
a

6.1476503169311405

In [19]:
a=[1,2,3,4,5]
exec("for i in a:print(i)")

1
2
3
4
5


In [22]:
globals = {'x':7,'y':10,'birds':['Parrot','Swallow','Albatross']}
locals={}

In [24]:
a = eval("3*x + 4*y",globals,locals)
a

61

In [25]:
exec("for b in birds:print(b)",globals,locals)

Parrot
Swallow
Albatross


In [27]:
s = "for i in range(10):print(i)"
c = compile(s,"",'exec')
exec(c)

0
1
2
3
4
5
6
7
8
9


In [29]:
x=1
y=2
s2 = "3*x + 4 * y"
c2 = compile(s2,'','eval')
eval(c2)

11